In [ ]:
from classiq import *
from math import radians

@qfunc
def main() -> None:
    """
    Joint Encryption + Error Correction demo using gate encoding
    following a simplified CSS + 3-stage quantum encryption protocol.
    """
    q = QArray[QBit]()
    allocate(8, q)  # Allocate 8 qubits

    # Step 1: Classical Bitstring (e.g., message to encode)
    classical_bits = [1, 0, 1, 0, 1]  # "10101" from the paper
    classical_bits += [0] * (8 - len(classical_bits))  # Pad to 8 bits

    # Step 2: CSS Encoding (simplified with Hadamard + X for redundancy)
    for i, bit in enumerate(classical_bits):
        if bit == 1:
            X(target=q[i])  # Encode 1 as |1⟩
        H(target=q[i])  # Add superposition

    # Step 3: Alice's Encryption (UA(θ) - secret rotations)
    for i in range(8):
        RZ(theta=radians(i * 15), target=q[i])  # Secret Rz rotation

    # Step 4: Bob's Transformation (UB(ϕ) - another layer)
    for i in range(8):
        RY(theta=radians(i * 10), target=q[i])  # Secret Ry rotation

    # Step 5: Alice's Decryption (inverse of her rotation)
    for i in range(8):
        RZ(theta=radians(-i * 15), target=q[i])  # Undo Rz

    # Step 6: Bob's Decryption (inverse of his rotation)
    for i in range(8):
        RY(theta=radians(-i * 10), target=q[i])  # Undo Ry

    # Step 7: Simulated Error Correction
    for i in range(8):
        if i % 2 == 0:
            Z(target=q[i])  # Simulated phase-flip correction
        else:
            X(target=q[i])  # Simulated bit-flip correction

# Build and run the model
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

job = execute(qprog)
print("Secure Quantum Communication Output:")
print(job.get_sample_result().parsed_counts)
